In [ ]:
import requests
import together
import json
from together import Together
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

In [ ]:
 TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")

In [ ]:

client = Together(api_key = TOGETHER_API_KEY)

     

In [ ]:

def get_crypto_price1(symbol):
    # url = 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    # https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest

    parameters = {
        'start': '1',
        'limit': '5000',
        'convert': 'USD'
    }
    
    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': 'a1f30c04-11ee-404e-8fca-40a7b9250fa7',  # Use your actual API key
    }
    
    session = requests.Session()
    session.headers.update(headers)
    
    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        # print(data)
        # Loop through the list of cryptocurrencies
        for crypto in data['data']:
            if crypto['symbol'] == symbol:  # Checking for the symbol
                return crypto['quote']['USD']['price']  # getting the price of the matched cryptocurrency
        
        return f"Cryptocurrency with symbol {symbol} not found."  # If symbol is not found

    except requests.exceptions.RequestException as e:
        return f"Error fetching price: {str(e)}"





In [ ]:
def translate(text):
  sys_msg = """You are a helpful assistant that can translate text from one language to another. Below are a few examples demonstrating how you 
  should translate sentences from one language to another. When a user asks for a translation, you should provide the equivalent sentence in the 
  target language. Just provide the translated sentence, nothing else.
  Examples :
  1. Translate from Spanish to English:
  User Input: "¿Cómo estás?"
  Agent Response: "How are you?"

  2. Translate from French to English:
  User Input: "Où est la bibliothèque?"
  Agent Response: "Where is the library?"

  3. Translate from German to English:
  User Input: "Ich habe Hunger."
  Agent Response: "I am hungry."

  """
  messages = [
    {"role": "system", "content": sys_msg},
    {"role": "user", "content": text}
  ]
  response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    messages=messages,
  )
  return response.choices[0].message.content


In [ ]:
import os
import json
import openai

# Initialize the OpenAI client
client1 = openai.OpenAI(
    base_url = "https://api.together.xyz/v1",
    api_key = TOGETHER_API_KEY,
)

# Tools definition
tools = [
  {
    "type": "function",
    "function": {
      "name": "get_crypto_price1",
      "description": "Get the current price of cryptocurrencies",
      "parameters": {
        "type": "object",
        "properties": {
          "symbol": {
            "type": "string",
            "description": "The symbol of the cryptocurrency",
          }
        }
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "translate",
      "description": "Translate text from one language to another.",
      "parameters": {
        "type": "object",
        "properties": {
          "text": {
            "type": "string",
            "description": "The text to be translated"
          }
        }
      }
    }
  }
]

# Define the system prompt
sys = """
You are a helpful assistant who helps people about getting the cryptocurrencies price and translating text. If the user asks a question about getting the 
cryptocurrencies price, first convert the cryptocurrency to its symbol and send the question to its respective tools.
Example:
Convert the Cryptocurrency Name to Its Symbol:
If the user says "Bitcoin", I convert it to "BTC".
If the user says "Ethereum", convert it to "ETH", and so on.
Request the Price:
Once the symbol is identified, call the respective tool to get the price using the symbol (e.g., BTC for Bitcoin, ETH for Ethereum).
Respond with the Price:
Then return the price information to the user in a user-friendly format.
"""

# Initialize messages list with the system message
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

while True:
    # Ask the user for a question
    question = input("Enter your question (type 'exit' to quit): ")

    # Check if the user wants to exit
    if question.lower() == 'exit':
        print("Goodbye!")
        break
    
    # Append the user question to messages
    messages.append({"role": "user", "content": question})
    try: 
        # Call the completion API with the conversation history
        response11 = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
        system_message = sys,
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )
    
        # Append assistant's response to the conversation history
        assistant_reply = response11.choices[0].message.content
        messages.append({"role": "assistant", "content": assistant_reply})
    
        # Print the assistant's response
        print(f"Assistant: {assistant_reply}")
    except Exception as e:
    # If an error happens, print a user-friendly message
        print("Sorry, something went wrong. Please try again.")
        print(f"Error: {str(e)}")  # You can print the actual error for debugging purposes
 
